In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

!conda install -c anaconda beautifulsoup4 -y
from bs4 import BeautifulSoup

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.0
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         143 KB

The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> conda-forge::ca-certificates-2020.12.5-ha878542_0
  certifi                anaconda::certifi-2020.6.20-py36_0 --> conda-forge::certifi-2020.12.5-py36h5fab9bb_1



certifi-2020.12.5    | 143 KB    | ###########################

In [2]:
# Loads the html into a pandas dataframe
australia = pd.read_csv('australian_postcodes.csv') # Loads the .csv into a pandas df

In [3]:
australia.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region,Lat_precise,Long_precise,SA1_MAINCODE_2011,SA1_MAINCODE_2016,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,RA_2011,RA_2016,MMM_2015,MMM_2019
0,230,200,ANU,ACT,149.119000,-35.277700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R1,-35.277700,149.119000,8.010510e+10,8.010510e+10,801051049.0,Acton,80105.0,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,NaN,NaN,NaN,R1,-35.277700,149.118527,8.010510e+10,8.010510e+10,801051049.0,Acton,80105.0,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,Darwin City,701.0,Darwin,R1,-12.393279,130.776661,7.010110e+10,7.010110e+10,701011002.0,Darwin City,70101.0,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,Darwin City,701.0,Darwin,R1,-12.463440,130.845642,7.010110e+10,7.010110e+10,701011002.0,Darwin City,70101.0,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,Darwin Suburbs,701.0,Darwin,R1,-12.432480,130.846254,7.010110e+10,7.010110e+10,701011007.0,Parap,70101.0,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0


In [4]:
 #new df for this analysis
nsw = australia[australia['postcode'] > 1999].reset_index(drop=True)
nsw2 = nsw[nsw['postcode']<2026].reset_index(drop=True)

In [5]:
nsw3 = nsw2[['postcode', 'locality', 'long','lat']]
nsw3.drop(nsw3.index[[10, 32]])
nsw3

,postcode,locality,long,lat
0,2000,BARANGAROO,151.201580,-33.860520
1,2000,DARLING HARBOUR,151.256649,-33.859953
2,2000,DAWES POINT,151.256649,-33.859953
3,2000,HAYMARKET,151.256649,-33.859953
4,2000,MILLERS POINT,151.256649,-33.859953
5,2000,PARLIAMENT HOUSE,151.256649,-33.859953
6,2000,SYDNEY,151.256649,-33.859953
7,2000,SYDNEY SOUTH,151.256649,-33.859953
8,2000,THE ROCKS,151.256649,-33.859953
9,2001,SYDNEY,151.268071,-33.794883


In [6]:
print('The dataframe has {} postcodes and {} localities.'.format(
        len(nsw3['postcode'].unique()),
        nsw3.shape[0]
    )
)

The dataframe has 23 postcodes and 57 localities.


In [7]:
address = 'Sydney, NSW, Australia'

geolocator = Nominatim(user_agent="syd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney are -33.8548157, 151.2164539.


In [38]:
# create map of Sydney using latitude and longitude values
map_syd = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, postcode, locality in zip(nsw3['lat'], nsw3['long'], nsw3['postcode'], nsw3['locality']):
    label = '{}, {}'.format(postcode, locality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_syd)  
    
map_syd

In [39]:
CLIENT_ID = 'WG0SDIXX4HJDEITR5WG5Z40ZEZKN1ORBTTECOZ0CWYXWVVD2' # your Foursquare ID
CLIENT_SECRET = 'EORG3STTH3DQF0HPMHTXIFJINCLFXD1HZ4Q0P40EQ2ERATJ5' # your Foursquare Secret
VERSION = '20210411' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WG0SDIXX4HJDEITR5WG5Z40ZEZKN1ORBTTECOZ0CWYXWVVD2
CLIENT_SECRET:EORG3STTH3DQF0HPMHTXIFJINCLFXD1HZ4Q0P40EQ2ERATJ5


In [40]:
nsw3.loc[0, 'locality']

'BARANGAROO'

In [41]:
neighborhood_latitude = nsw3.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = nsw3.loc[0, 'long'] # neighborhood longitude value

neighborhood_name = nsw3.loc[0, 'locality'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of BARANGAROO are -33.86051951, 151.2015802.


In [42]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 750 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WG0SDIXX4HJDEITR5WG5Z40ZEZKN1ORBTTECOZ0CWYXWVVD2&client_secret=EORG3STTH3DQF0HPMHTXIFJINCLFXD1HZ4Q0P40EQ2ERATJ5&v=20210411&ll=-33.86051951,151.2015802&radius=750&limit=50'

In [43]:
results = requests.get(url).json()

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(40)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Langham Hotel Sydney,Hotel,-33.860517,151.203437
1,Palisade Hotel,Pub,-33.857979,151.202264
2,Lord Nelson Brewery Hotel,Brewery,-33.858403,151.203548
3,Fish at the Rocks,Seafood Restaurant,-33.858673,151.203500
4,Observatory Hill,Park,-33.859125,151.204977
5,Sydney Observatory,Planetarium,-33.859534,151.204643
6,Barangaroo Reserve,Park,-33.857052,151.201100
7,Bourke Street Bakery,Bakery,-33.864570,151.201480
8,CAVA,Coffee Shop,-33.862581,151.204053
9,Blu Bar On 36,Hotel Bar,-33.861067,151.206361


In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


In [47]:
def getNearbyVenues(names, latitude, longitude, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitude, longitude):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng,radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"].get('groups',[{}])[0].get('items', [])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
# type your answer here
syd_venues = getNearbyVenues(names=nsw3['locality'],longitude=nsw3['long'],latitude=nsw3['lat'])

BARANGAROO
DARLING HARBOUR
DAWES POINT
HAYMARKET
MILLERS POINT
PARLIAMENT HOUSE
SYDNEY
SYDNEY SOUTH
THE ROCKS
SYDNEY
WORLD SQUARE
ALEXANDRIA MC
EASTERN SUBURBS MC
THE UNIVERSITY OF SYDNEY
BROADWAY
ULTIMO
CHIPPENDALE
DARLINGTON
GOLDEN GROVE
DARLING ISLAND
PYRMONT
DARLINGHURST
SURRY HILLS
TAYLOR SQUARE
ELIZABETH BAY
HMAS KUTTABUL
KINGS CROSS
POTTS POINT
RUSHCUTTERS BAY
WOOLLOOMOOLOO
STRAWBERRY HILLS
STRAWBERRY HILLS
ALEXANDRIA
BEACONSFIELD
EVELEIGH
REDFERN
WATERLOO
WATERLOO DC
ZETLAND
EASTLAKES
ROSEBERY
BANKSMEADOW
BOTANY
MASCOT
SYDNEY DOMESTIC AIRPORT
SYDNEY INTERNATIONAL AIRPORT
CENTENNIAL PARK
MOORE PARK
PADDINGTON
BONDI JUNCTION
BONDI JUNCTION PLAZA
QUEENS PARK
BELLEVUE HILL
BRONTE
CHARING CROSS
WAVERLEY
WOOLLAHRA


In [49]:
print(syd_venues.shape)
syd_venues.head()

(1902, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BARANGAROO,-33.86052,151.20158,The Langham Hotel Sydney,-33.860517,151.203437,Hotel
1,BARANGAROO,-33.86052,151.20158,Palisade Hotel,-33.857979,151.202264,Pub
2,BARANGAROO,-33.86052,151.20158,Lord Nelson Brewery Hotel,-33.858403,151.203548,Brewery
3,BARANGAROO,-33.86052,151.20158,Fish at the Rocks,-33.858673,151.203500,Seafood Restaurant
4,BARANGAROO,-33.86052,151.20158,Observatory Hill,-33.859125,151.204977,Park


In [50]:
syd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALEXANDRIA,48,48,48,48,48,48
ALEXANDRIA MC,36,36,36,36,36,36
BANKSMEADOW,6,6,6,6,6,6
BARANGAROO,50,50,50,50,50,50
BEACONSFIELD,48,48,48,48,48,48
BELLEVUE HILL,8,8,8,8,8,8
BONDI JUNCTION,41,41,41,41,41,41
BONDI JUNCTION PLAZA,41,41,41,41,41,41
BOTANY,6,6,6,6,6,6


In [51]:
print('There are {} uniques categories.'.format(len(syd_venues['Venue Category'].unique())))

There are 187 uniques categories.


In [52]:
# one hot encoding
syd_onehot = pd.get_dummies(syd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
syd_onehot['Neighborhood'] = syd_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [syd_onehot.columns[-1]] + list(syd_onehot.columns[:-1])
syd_onehot = syd_onehot[fixed_columns]

syd_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Boat or Ferry,Bookstore,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Car Wash,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Cricket Ground,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,Office,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,BARANGAROO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BARANGAROO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,BARANGAROO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,BARANGAROO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [53]:
syd_onehot.shape

(1902, 188)

In [54]:
syd_grouped = syd_onehot.groupby('Neighborhood').mean().reset_index()
syd_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Boat or Ferry,Bookstore,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Car Wash,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Cricket Ground,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,Office,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALEXANDRIA,0.0,0.0,0.0,0.0,0.0,0.020833,0.020833,0.0,0.0,0.00,0.0,0.0,0.000000,0.020833,0.0,0.0,0.020833,0.000000,0.0,0.000000,0.020833,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.020833,0.0,0.041667,0.0,0.000000,0.0,0.270833,0.020833,0.0,0.00,0.0,0.0,0.00,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.00,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.020833,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.041667,0.00,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.020833,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.00,0.041667,0.0,0.020833,0.00,0.0,0.020833,0.0,0.0,0.0,0.000000,0.0,0.0,0.020833,0.0,0.00,0.0,0.0,0.0,0.020833,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.0,0.020833,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALEXANDRIA MC,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.027778,0.027778,0.0,0.0,0.000000,0.000000,0.0,0.027778,0.027778,0.0,0.027778,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.000000,0.0,0.027778,0.0,0.027778,0.0,0.305556,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00

In [55]:
syd_grouped.shape

(54, 188)

In [56]:
num_top_venues = 5

for hood in syd_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = syd_grouped[syd_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALEXANDRIA----
                venue  freq
0                Café  0.27
1             Brewery  0.04
2  Italian Restaurant  0.04
3         Coffee Shop  0.04
4           Pet Store  0.04


----ALEXANDRIA MC----
                venue  freq
0                Café  0.31
1          Playground  0.06
2       Bowling Green  0.03
3  Basketball Stadium  0.03
4     Thai Restaurant  0.03


----BANKSMEADOW----
             venue  freq
0     Antique Shop  0.17
1      Coffee Shop  0.17
2           Bakery  0.17
3             Park  0.17
4  Badminton Court  0.17


----BARANGAROO----
         venue  freq
0         Café  0.10
1        Hotel  0.08
2          Pub  0.08
3       Bakery  0.06
4  Coffee Shop  0.06


----BEACONSFIELD----
                venue  freq
0                Café  0.27
1             Brewery  0.04
2  Italian Restaurant  0.04
3         Coffee Shop  0.04
4           Pet Store  0.04


----BELLEVUE HILL----
                venue  freq
0                Park  0.38
1                Café  0.25
2  

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = syd_grouped['Neighborhood']

for ind in np.arange(syd_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(syd_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRIA,Café,Brewery,Pet Store,Coffee Shop,Miscellaneous Shop,Italian Restaurant,Furniture / Home Store,Electronics Store,Playground,Basketball Stadium
1,ALEXANDRIA MC,Café,Playground,Sandwich Place,Bar,Supermarket,Basketball Stadium,Bowling Green,Shipping Store,Seafood Restaurant,Brewery
2,BANKSMEADOW,Café,Coffee Shop,Antique Shop,Bakery,Badminton Court,Park,Dive Bar,Fast Food Restaurant,Farmers Market,Event Space
3,BARANGAROO,Café,Pub,Hotel,Seafood Restaurant,Coffee Shop,Bar,Bakery,Park,Chinese Restaurant,Flea Market
4,BEACONSFIELD,Café,Brewery,Pet Store,Coffee Shop,Miscellaneous Shop,Italian Restaurant,Furniture / Home Store,Electronics Store,Playground,Basketball Stadium


In [59]:
# set number of clusters
kclusters = 5

syd_grouped_clustering = syd_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(syd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 3, 0, 4, 3, 3, 4, 3], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

syd_merged = nsw3

# merge syd_grouped with nsw3 to add latitude/longitude for each neighborhood
syd_merged = syd_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='locality', sort = False)

syd_merged.head() # check the last columns!

,postcode,locality,long,lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2000,BARANGAROO,151.201580,-33.860520,3.0,Café,Pub,Hotel,Seafood Restaurant,Coffee Shop,Bar,Bakery,Park,Chinese Restaurant,Flea Market
1,2000,DARLING HARBOUR,151.256649,-33.859953,1.0,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
2,2000,DAWES POINT,151.256649,-33.859953,1.0,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
3,2000,HAYMARKET,151.256649,-33.859953,1.0,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
4,2000,MILLERS POINT,151.256649,-33.859953,1.0,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop


In [61]:
syd_merged['Cluster Labels'] = syd_merged['Cluster Labels'].fillna(0).astype(int)
syd_merged2 = syd_merged.drop([30, 31, 37])#remove rows which would have NaN from analysis
syd_merged2.head()

,postcode,locality,long,lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2000,BARANGAROO,151.201580,-33.860520,3,Café,Pub,Hotel,Seafood Restaurant,Coffee Shop,Bar,Bakery,Park,Chinese Restaurant,Flea Market
1,2000,DARLING HARBOUR,151.256649,-33.859953,1,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
2,2000,DAWES POINT,151.256649,-33.859953,1,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
3,2000,HAYMARKET,151.256649,-33.859953,1,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
4,2000,MILLERS POINT,151.256649,-33.859953,1,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(syd_merged2['lat'], syd_merged2['long'], syd_merged2['locality'], syd_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
syd_merged2.loc[syd_merged2['Cluster Labels'] == 0, syd_merged2.columns[[1] + list(range(5, syd_merged2.shape[1]))]]

,locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,SYDNEY,Café,Park,Harbor / Marina,Japanese Restaurant,Hardware Store,Furniture / Home Store,Shopping Mall,Farmers Market,Supermarket,Liquor Store
9,SYDNEY,Café,Park,Harbor / Marina,Japanese Restaurant,Hardware Store,Furniture / Home Store,Shopping Mall,Farmers Market,Supermarket,Liquor Store
11,ALEXANDRIA MC,Café,Playground,Sandwich Place,Bar,Supermarket,Basketball Stadium,Bowling Green,Shipping Store,Seafood Restaurant,Brewery
16,CHIPPENDALE,Café,Bar,Pub,Coffee Shop,Bakery,Farmers Market,Beer Bar,Market,Dive Bar,Ramen Restaurant
17,DARLINGTON,Café,Bar,Pub,Coffee Shop,Bakery,Farmers Market,Beer Bar,Market,Dive Bar,Ramen Restaurant
18,GOLDEN GROVE,Café,Bar,Pub,Coffee Shop,Bakery,Farmers Market,Beer Bar,Market,Dive Bar,Ramen Restaurant
21,DARLINGHURST,Café,Coffee Shop,Pub,Yoga Studio,Sandwich Place,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bar,Pizza Place,Gym
22,SURRY HILLS,Café,Coffee Shop,Pub,Yoga Studio,Sandwich Place,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bar,Pizza Place,Gym
23,TAYLOR SQUARE,Café,Coffee Shop,Pub,Yoga Studio,Sandwich Place,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bar,Pizza Place,Gym
32,ALEXANDRIA,Café,Brewery,Pet Store,Coffee Shop,Miscellaneous Shop,Italian Restaurant,Furniture / Home Store,Electronics Store,Playground,Basketball Stadium


In [64]:
syd_merged2.loc[syd_merged2['Cluster Labels'] == 1, syd_merged2.columns[[1] + list(range(5, syd_merged2.shape[1]))]]

,locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,DARLING HARBOUR,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
2,DAWES POINT,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
3,HAYMARKET,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
4,MILLERS POINT,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
5,PARLIAMENT HOUSE,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
7,SYDNEY SOUTH,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
8,THE ROCKS,Harbor / Marina,Park,Beach,Discount Store,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop


In [65]:
syd_merged2.loc[syd_merged2['Cluster Labels'] == 2, syd_merged2.columns[[1] + list(range(5, syd_merged2.shape[1]))]]

,locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,MASCOT,Airport Lounge,Airport Service,Fast Food Restaurant,Café,Coffee Shop,Donut Shop,Beer Bar,Tapas Restaurant,Women's Store,Juice Bar
44,SYDNEY DOMESTIC AIRPORT,Airport Lounge,Airport Service,Fast Food Restaurant,Café,Coffee Shop,Donut Shop,Beer Bar,Tapas Restaurant,Women's Store,Juice Bar
45,SYDNEY INTERNATIONAL AIRPORT,Airport Lounge,Airport Service,Fast Food Restaurant,Café,Coffee Shop,Donut Shop,Beer Bar,Tapas Restaurant,Women's Store,Juice Bar


In [66]:
syd_merged2.loc[syd_merged2['Cluster Labels'] == 3, syd_merged2.columns[[1] + list(range(5, syd_merged2.shape[1]))]]

,locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BARANGAROO,Café,Pub,Hotel,Seafood Restaurant,Coffee Shop,Bar,Bakery,Park,Chinese Restaurant,Flea Market
10,WORLD SQUARE,Thai Restaurant,Japanese Restaurant,Hotel,Café,Korean BBQ Restaurant,Malay Restaurant,Chinese Restaurant,Coffee Shop,Burger Joint,Breakfast Spot
13,THE UNIVERSITY OF SYDNEY,Café,Thai Restaurant,Pub,Vietnamese Restaurant,Pizza Place,Italian Restaurant,Coffee Shop,Cosmetics Shop,Sandwich Place,Fast Food Restaurant
14,BROADWAY,Café,Coffee Shop,Thai Restaurant,Bar,Pub,Dessert Shop,Hotel,Supermarket,Shopping Mall,Bookstore
15,ULTIMO,Café,Coffee Shop,Thai Restaurant,Bar,Pub,Dessert Shop,Hotel,Supermarket,Shopping Mall,Bookstore
19,DARLING ISLAND,Café,Pub,Fish Market,Bar,Seafood Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,French Restaurant,Australian Restaurant
20,PYRMONT,Café,Pub,Fish Market,Bar,Seafood Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,French Restaurant,Australian Restaurant
24,ELIZABETH BAY,Café,Italian Restaurant,Australian Restaurant,Pub,Coffee Shop,Wine Bar,Gym,Lounge,Japanese Restaurant,Speakeasy
25,HMAS KUTTABUL,Café,Italian Restaurant,Australian Restaurant,Pub,Coffee Shop,Wine Bar,Gym,Lounge,Japanese Restaurant,Speakeasy
26,KINGS CROSS,Café,Italian Restaurant,Australian Restaurant,Pub,Coffee Shop,Wine Bar,Gym,Lounge,Japanese Restaurant,Speakeasy


In [67]:
syd_merged2.loc[syd_merged2['Cluster Labels'] == 4, syd_merged2.columns[[1] + list(range(5, syd_merged2.shape[1]))]]

,locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,EASTERN SUBURBS MC,Bakery,Park,Sandwich Place,Coffee Shop,Badminton Court,Café,Discount Store,Event Space,Electronics Store,Dumpling Restaurant
41,BANKSMEADOW,Café,Coffee Shop,Antique Shop,Bakery,Badminton Court,Park,Dive Bar,Fast Food Restaurant,Farmers Market,Event Space
42,BOTANY,Café,Coffee Shop,Antique Shop,Bakery,Badminton Court,Park,Dive Bar,Fast Food Restaurant,Farmers Market,Event Space
52,BELLEVUE HILL,Park,Café,Pizza Place,Italian Restaurant,Bagel Shop,Discount Store,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant
